# Optogenetic Response Analysis

This notebook demonstrates how to analyze fly responses to optogenetic stimulation.

## What you'll learn

1. Analyzing event-triggered responses
2. Filtering results by response and metadata
3. Visualizing response traces
4. Comparing conditions (e.g., intensity levels, sham vs real)

In [ ]:
import braidz_analysis as ba
import matplotlib.pyplot as plt
import numpy as np

print(f"braidz_analysis version: {ba.__version__}")

## 1. Load Data

Load braidz files containing optogenetic stimulation data.

In [ ]:
# === CONFIGURE YOUR DATA PATH HERE ===
DATA_PATH = "/path/to/your/experiments"  # Update this!
BRAIDZ_FILE = "opto_experiment.braidz"   # Update this!

# Load data
data = ba.read_braidz(BRAIDZ_FILE, base_folder=DATA_PATH)

print(data)
print(f"\nOpto events available: {data.has_opto}")

if data.has_opto:
    print(f"Number of opto events: {len(data.opto)}")
    print(f"Opto columns: {data.opto.columns.tolist()}")

## 2. Analyze Event Responses

Use `analyze_event_responses()` to extract response data around each optogenetic event.

In [ ]:
# Analyze optogenetic responses with default config
opto_results = ba.analyze_event_responses(
    data.trajectories,
    data.opto,
    progressbar=True
)

print(opto_results)
print(f"\nResponse rate: {opto_results.response_rate:.1%}")

In [ ]:
# Examine the results structure
print("Traces available:")
for key, arr in opto_results.traces.items():
    print(f"  {key}: {arr.shape}")

print("\nMetrics columns:")
print(opto_results.metrics.columns.tolist())

print("\nMetadata columns:")
print(opto_results.metadata.columns.tolist())

In [ ]:
# Preview metrics
opto_results.metrics.head()

## 3. Filtering Results

Filter results by response status or metadata.

In [ ]:
# Get responsive and non-responsive trials
responsive = opto_results.responsive
non_responsive = opto_results.non_responsive

print(f"Responsive trials: {len(responsive)}")
print(f"Non-responsive trials: {len(non_responsive)}")

In [ ]:
# Filter by sham (if available)
if 'sham' in opto_results.metadata.columns:
    real_trials = opto_results.real
    sham_trials = opto_results.sham
    
    print(f"Real trials: {len(real_trials)} (response rate: {real_trials.response_rate:.1%})")
    print(f"Sham trials: {len(sham_trials)} (response rate: {sham_trials.response_rate:.1%})")

In [ ]:
# Filter by custom criteria
# Example: Filter by intensity (if available)
if 'intensity' in opto_results.metadata.columns:
    intensities = opto_results.metadata['intensity'].unique()
    print(f"Available intensities: {sorted(intensities)}")
    
    for intensity in sorted(intensities):
        subset = opto_results.filter(intensity=intensity)
        print(f"  Intensity {intensity}: {len(subset)} trials, {subset.response_rate:.1%} response rate")

## 4. Visualizing Response Traces

Plot angular and linear velocity traces around the stimulus.

In [ ]:
# Create summary figure
fig, axes = ba.create_summary_figure(
    opto_results,
    title="All Optogenetic Trials"
)
plt.show()

In [ ]:
# Compare responsive vs non-responsive
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Angular velocity
ba.plot_angular_velocity(responsive, ax=axes[0], color='tab:blue', label='Responsive')
ba.plot_angular_velocity(non_responsive, ax=axes[0], color='tab:gray', label='Non-responsive')
axes[0].legend()
axes[0].set_title('Angular Velocity')

# Heading change distribution
ba.plot_heading_distribution(responsive, ax=axes[1], color='tab:blue', alpha=0.5, label='Responsive')
ba.plot_heading_distribution(non_responsive, ax=axes[1], color='tab:gray', alpha=0.5, label='Non-resp')
axes[1].legend()
axes[1].set_title('Heading Change Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Convert x-axis to milliseconds
fig, ax = plt.subplots(figsize=(8, 4))
ba.plot_angular_velocity(responsive, ax=ax)
ba.convert_frames_to_ms(ax, fps=100, tick_step_ms=100)
ax.set_title('Angular Velocity Response')
plt.show()

## 5. Comparing Conditions

Compare responses across different experimental conditions.

In [ ]:
# Compare real vs sham with violin plots
if 'sham' in opto_results.metadata.columns:
    fig, ax = plt.subplots(figsize=(8, 4))
    
    ba.plot_heading_comparison(
        groups=['Real', 'Sham'],
        results_list=[real_trials.responsive, sham_trials.responsive],
        ax=ax
    )
    ax.set_title('Heading Change: Real vs Sham (Responsive only)')
    plt.show()

In [ ]:
# Plot response rate by intensity (if available)
if 'intensity' in opto_results.metadata.columns:
    fig, ax = plt.subplots(figsize=(8, 4))
    ba.plot_response_rate_by_group(opto_results.real, group_by='intensity', ax=ax)
    ax.set_title('Response Rate by Intensity')
    plt.show()

## 6. Examining Individual Trials

Visualize specific trajectories around the stimulus event.

In [ ]:
# Plot trajectory for a responsive trial
if len(responsive) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Find trial with largest heading change
    idx = np.nanargmax(np.abs(responsive.metrics['heading_change'].values))
    
    # Trajectory in x-y plane
    ba.plot_trajectory(
        responsive, index=idx, dims=('x', 'y'),
        ax=axes[0], highlight_range=(50, 80)
    )
    axes[0].set_title('X-Y Trajectory')
    
    # Angular velocity trace for this trial
    ax = axes[1]
    omega = np.degrees(responsive.traces['angular_velocity'][idx])
    ax.plot(omega)
    ba.add_stimulus_region(ax, 50, 80, color='tab:red')
    ax.set_xlabel('Frames')
    ax.set_ylabel('Angular Velocity (deg/s)')
    ax.set_title('Angular Velocity')
    
    # Linear velocity trace
    ax = axes[2]
    speed = responsive.traces['linear_velocity'][idx]
    ax.plot(speed)
    ba.add_stimulus_region(ax, 50, 80, color='tab:red')
    ax.set_xlabel('Frames')
    ax.set_ylabel('Linear Velocity (m/s)')
    ax.set_title('Linear Velocity')
    
    hc = np.degrees(responsive.metrics['heading_change'].iloc[idx])
    rt = responsive.metrics['reaction_time'].iloc[idx]
    fig.suptitle(f'Trial with heading change = {hc:.1f}°, reaction time = {rt:.0f} frames')
    
    plt.tight_layout()
    plt.show()

## 7. Custom Configuration

Adjust analysis parameters for different experimental designs.

In [ ]:
# Use custom configuration
custom_config = ba.Config(
    response_window=50,      # Longer window to detect response
    saccade_threshold=250,   # Lower threshold for saccade detection
    pre_frames=100,          # More context before stimulus
    post_frames=150,         # More context after stimulus
)

custom_results = ba.analyze_event_responses(
    data.trajectories,
    data.opto,
    config=custom_config,
    progressbar=True
)

print(f"Response rate with default config: {opto_results.response_rate:.1%}")
print(f"Response rate with custom config: {custom_results.response_rate:.1%}")

## 8. Computing Statistics

Generate summary statistics for reporting.

In [ ]:
# Compute response statistics grouped by metadata
stats = ba.compute_response_statistics(opto_results.real)
stats

In [ ]:
# Manual statistics
resp = responsive
print("Response Metrics Summary (Responsive trials only):")
print(f"  Heading change (abs): {np.nanmean(np.abs(resp.metrics['heading_change'])):.2f} ± "
      f"{np.nanstd(np.abs(resp.metrics['heading_change'])):.2f} rad")
print(f"  Heading change (abs): {np.degrees(np.nanmean(np.abs(resp.metrics['heading_change']))):.1f} ± "
      f"{np.degrees(np.nanstd(np.abs(resp.metrics['heading_change']))):.1f} deg")
print(f"  Reaction time: {np.nanmean(resp.metrics['reaction_time']):.1f} ± "
      f"{np.nanstd(resp.metrics['reaction_time']):.1f} frames")
print(f"  Peak velocity: {np.degrees(np.nanmean(np.abs(resp.metrics['peak_velocity']))):.1f} ± "
      f"{np.degrees(np.nanstd(np.abs(resp.metrics['peak_velocity']))):.1f} deg/s")

## Summary

In this notebook, you learned how to:

1. Analyze optogenetic responses using `ba.analyze_event_responses()`
2. Filter results using `.responsive`, `.real`, `.filter()`
3. Visualize traces using `ba.plot_angular_velocity()` and `ba.create_summary_figure()`
4. Compare conditions using `ba.plot_heading_comparison()`
5. Examine individual trials using `ba.plot_trajectory()`
6. Customize analysis with `ba.Config`

**Note:** The same workflow works for visual stimulus analysis - just use `data.stim` instead of `data.opto`.